Vamos a tomar dos medidas para ver la coherencia entre dos particiones:
* **Presición** usando algo similar a la [matriz confusion](https://es.wikipedia.org/wiki/Matriz_de_confusi%C3%B3n)
* [**Información mutua**](https://es.wikipedia.org/wiki/Informaci%C3%B3n_mutua) que mide la dependencia mutua de las dos variables, es decir, mide la reducción de la incertidumbre (entropía) de una variable X debido al conocimiento del valor de otra variable de referencia Y.

El primer método consiste en buscar la presición de la nueva partición (test) para eso consideraremos que hay un "error" en la nueva partición cuando se cumplan dos cosas:

* Dos nodos que pertenecen a la misma comunidad de referencia son
asignados a comunidades diferentes
* Dos nodos de diferentes comunidades de referencia son asignados a una
misma comunidad

Para ver la presición nos quedaremos con los casos donde no hubo error. Para eso buscamos dos valores: los nodos que siguen estando en la nueva partición juntos y los nodos que siguen separados. 

In [5]:
def matriz_confusion(test, control, cantidad_de_nodos):
    """
    Función que devuelve aciertos de una partición tomando otra como control.
    
    test: array de clusters de nueva partición
    control: array de clusters de partición con la que comparar
    """
    
    mismo_mismo = 0
    mismo_distinto = 0
    distinto_mismo = 0
    distinto_distinto = 0
    
    for par in combinations(range(0, cantidad_de_nodos), 2):
        idx_1, idx_2 = par
        if test[idx_1] == test[idx_2] and control[idx_1] == control[idx_2]:
            mismo_mismo += 1 
        if not test[idx_1] == test[idx_2] and not control[idx_1] == control[idx_2]:
            distinto_distinto += 1
    
    return mismo_mismo + distinto_distinto

def precision(test, control):
    cantidad_de_nodos = len(test)
    aciertos = matriz_confusion(test, control, cantidad_de_nodos)
    pares_totales = cantidad_de_nodos * (cantidad_de_nodos - 1) / 2
    return aciertos / pares_totales

La otra medida que usaremos será información mutua. Para lo mismo usaremos la función [normalized_mutual_info_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html) de la libreria Scikit-Learn. 

In [6]:
from sklearn.metrics import normalized_mutual_info_score as info_mutua

Me importo las particiones halladas en el inciso **(a)**. Hay cuatro métodos, por cada uno tenemos un diccionario que tiene de *keys* a los nodos y como *value* su comunidad.

In [3]:
%store -r particiones

Queremos convertir el diccionario de cada método en un array. 

In [22]:
particiones_listas = {}

for metodo in particiones:
    particiones_listas[metodo] = list(particiones[metodo].values())

In [23]:
from itertools import combinations

comparaciones = {}
metodos = particiones.keys()

for m_1, m_2 in list(combinations(metodos, 2)):
    par = (m_1, m_2)
    comparaciones[par] = {}
    test = particiones_listas[m_2]
    control = particiones_listas[m_1] 
    comparaciones[par]["Precision"] = precision(test, control)
    comparaciones[par]['Info_mutua'] = info_mutua(test, control) #aritmethic es default

In [24]:
import pandas as pd 

df_coherencia = pd.DataFrame(comparaciones)
df_coherencia.round(3)

Edge_bet                  Fast_greedy         Louvline
           Fast_greedy Louvline Infomap    Louvline Infomap  Infomap
Info_mutua       0.811    0.068   0.672       0.044   0.689    0.069
Precision        0.891    0.617   0.840       0.587   0.841    0.647

La **información mutua** de dos variables es una cantidad que mide la dependencia mutua de las dos variables. Intuitivamente, la información mutua mide la información que X e Y comparten: mide en cuánto el conocimiento de una variable reduce nuestra incertidumbre sobre la otra. 

Por ejemplo, si X e Y son independientes, entonces conocer X no da información sobre Y y viceversa, por lo que su información mutua es cero. En el otro extremo, si X e Y son idénticas entonces toda información proporcionada por X es compartida por Y: saber X determina el valor de Y y viceversa.

La **precisión** nos da la proporción de aciertos (cuantos delfines fueron correctamente separados o puestos en la misma comunidad) según la cantidad total de pares de delfines. Se aclara que se considera _correcta_ a la partición que se toma como referencia _control_. El valor de la misma es más bajo ~60% de de "aciertos" para Louvline, mientras que las demás particiones tienen valores más altos que el 80%. 